#### Import Libraries

In [2]:
import pandas as pd
from openai import OpenAI
import json

### Initialize OpenAI

In [3]:
openai_client = OpenAI(api_key='YOUR_KEY_HERE')

### Load the pre-existing symptoms database

In [4]:
data = pd.read_json("./intents.json")

In [5]:
data.head(50)

,intents
0,"{'tag': 'Cuts', 'patterns': ['What to do if Cu..."
1,"{'tag': 'Abrasions', 'patterns': ['how do you ..."
2,"{'tag': 'stings', 'patterns': ['How do you tre..."
3,"{'tag': 'Splinter', 'patterns': ['How to remov..."
4,"{'tag': 'Sprains', 'patterns': ['How do you tr..."
5,"{'tag': 'Strains', 'patterns': ['How do you tr..."
6,"{'tag': 'Fever', 'patterns': ['How do you trea..."
7,"{'tag': 'Nasal Congestion', 'patterns': ['How ..."
8,"{'tag': 'Cough', 'patterns': ['How to cure cou..."
9,"{'tag': 'Sore Throat', 'patterns': ['How do yo..."


In [6]:
data = data['intents']

In [7]:
len(data)

44

In [8]:
train_data = pd.DataFrame(columns=['intent','patterns','response'])

### Making the data format bit readable

In [9]:
for index in data:
        new_data = {'intent': index['tag'].lower(),'patterns': index['patterns'],'response':index['responses']}
        train_data = train_data._append(new_data, ignore_index=True)

First prompt to begin the conversation in the bot

In [10]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    health_issues_category = [
        'Cuts', 'Abrasions', 'Stings', 'Splinter', 'Sprains', 'Strains', 'Fever',
        'Nasal Congestion', 'Cough', 'Sore Throat', 'Gastrointestinal problems',
        'Skin problems', 'Abdominal Pain', 'Bruises', 'Broken Toe', 'Choking',
        'Wound', 'Diarrhea', 'Frostbite', 'Heat Exhaustion', 'Heat Stroke',
        'Insect Bites', 'Nose Bleed', 'Pulled Muscle', 'Rectal Bleeding', 'Sun Burn',
        'Testicle Pain', 'Vertigo', 'Normal Bleeding', 'Eye Injury', 'Chemical Burn',
        'Poison', 'Teeth Issues', 'Seizure', 'Head Injury', 'Fainting', 'Headache',
        'Cold', 'Rash', 'Snake Bite', 'Animal Bite', 'Drowning', 'CPR', 'Fracture'
    ]
    
    response_format = {'issue': '_'}

    system_message = f"""
    You are an intelligent healthcare expert. Your goal is to identify the patient's health issue by asking relevant questions and analyzing their responses. 
    You must categorize the health issue based on the following categories: {health_issues_category}. Any response that does not fit these categories will be penalized.
    The final response should be in JSON format, following this structure: {response_format}, with the 'issue' key filled in accordingly.

    Follow these steps to understand and identify the user's health issue:
    {delimiter}
    Thought 1: Ask initial questions to understand the user's health issue. If the issue is unclear, ask follow-up questions for more details. Ensure the issue falls within the {health_issues_category} categories.
    {delimiter}
    Thought 2: If the user is unclear, ask indirect questions related to the {health_issues_category}. Rephrase questions if necessary to capture the problem clearly.
    {delimiter}

    Example conversation:
    User: "Hi, I am facing health issues."
    Assistant: "I'm sorry to hear that. Can you describe your symptoms a bit more? Do you have a runny nose or a cough? Please tell me more so I can assist you better."
    User: "I think I have a cough, but I'm not sure."
    Assistant: "Thank you for the information. Let me help you further."
    Assistant: "{response_format}"
    {delimiter}

    Begin with a short welcome message and encourage the user to share their health issues.
    """
    
    conversation = [{"role": "system", "content": system_message}]
    return conversation


In [11]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent healthcare expert. Your goal is to identify the patient\'s health issue by asking relevant questions and analyzing their responses. \n    You must categorize the health issue based on the following categories: [\'Cuts\', \'Abrasions\', \'Stings\', \'Splinter\', \'Sprains\', \'Strains\', \'Fever\', \'Nasal Congestion\', \'Cough\', \'Sore Throat\', \'Gastrointestinal problems\', \'Skin problems\', \'Abdominal Pain\', \'Bruises\', \'Broken Toe\', \'Choking\', \'Wound\', \'Diarrhea\', \'Frostbite\', \'Heat Exhaustion\', \'Heat Stroke\', \'Insect Bites\', \'Nose Bleed\', \'Pulled Muscle\', \'Rectal Bleeding\', \'Sun Burn\', \'Testicle Pain\', \'Vertigo\', \'Normal Bleeding\', \'Eye Injury\', \'Chemical Burn\', \'Poison\', \'Teeth Issues\', \'Seizure\', \'Head Injury\', \'Fainting\', \'Headache\', \'Cold\', \'Rash\', \'Snake Bite\', \'Animal Bite\', \'Drowning\', \'CPR\', \'Fracture\']. Any response that does not fit these categori

In [12]:
print(debug_conversation[0]['content'])


    You are an intelligent healthcare expert. Your goal is to identify the patient's health issue by asking relevant questions and analyzing their responses. 
    You must categorize the health issue based on the following categories: ['Cuts', 'Abrasions', 'Stings', 'Splinter', 'Sprains', 'Strains', 'Fever', 'Nasal Congestion', 'Cough', 'Sore Throat', 'Gastrointestinal problems', 'Skin problems', 'Abdominal Pain', 'Bruises', 'Broken Toe', 'Choking', 'Wound', 'Diarrhea', 'Frostbite', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Nose Bleed', 'Pulled Muscle', 'Rectal Bleeding', 'Sun Burn', 'Testicle Pain', 'Vertigo', 'Normal Bleeding', 'Eye Injury', 'Chemical Burn', 'Poison', 'Teeth Issues', 'Seizure', 'Head Injury', 'Fainting', 'Headache', 'Cold', 'Rash', 'Snake Bite', 'Animal Bite', 'Drowning', 'CPR', 'Fracture']. Any response that does not fit these categories will be penalized.
    The final response should be in JSON format, following this structure: {'issue': '_'}, with the '

### OpenAI call for the chat completions

In [13]:
# Define a Chat Completions API call
def get_chat_completions(input):
        MODEL = 'gpt-3.5-turbo'
        chat_completion = openai_client.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)
        output = chat_completion.choices[0].message.content
        return output

In [14]:
input_prompt ='What is the capital of France?'
messages = [{'role':'user','content':input_prompt}]

In [15]:
get_chat_completions(messages)

'The capital of France is Paris.'

### Moderation check for the bad messages

In [16]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai_client.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

### Find out the solution as described by the doctors for the health issues

In [17]:
def suggest_health_solution(intent):
    data = None
    try:
        json_data = json.loads(intent)
        if 'issue' in json_data:            
            checkIntent = json_data['issue'].lower() in train_data['intent'].values
            if(checkIntent):
                #we found the intent from chatbot, lets retrieve and show the result
                data_1 = train_data[train_data['intent'] == json_data['issue'].lower()]['response']
                data = data_1[data_1.index[0]][0]
                return data
        else:
            return data
    except ValueError as e:
        print(e)
        return data

### check if dictionary is present in the text and extract it 

In [18]:
def dictionary_present(response):
    delimiter = "####"

    data_format = {"issue": "cough"}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {data_format}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format with double quotes in keys and values.
            The output should match the format as {data_format}.

            {delimiter}
            The output should contain the exact keys and values as present in the input.
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - Thank you for clarifying. Based on your symptom of running nose without sore throat or cough, it seems like you may have Nasal Congestion. {{'issue': 'Nasal Congestion'}}
            output 1: {{"issue": "Nasal Congestion"}}

            input 2: - Thank you for clarifying. Based on your symptom of fever without sore throat or cough, it seems like you may have fever. {{'issue': 'Fever'}}
            output 2: {{"issue": "Fever"}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages)

    return confirmation

In [19]:
string_dict = "Thank you for clarifying. Based on your symptom of running nose without sore throat or cough, it seems like you may have Nasal Congestion.{'issue': 'Nasal Congestion'}"""
check_dict = dictionary_present(string_dict)

In [20]:
check_dict

'{"issue": "Nasal Congestion"}'

### Dialogue management system for the bot

In [21]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')
    
    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
            
        conversation.append({"role": "user", "content": user_input})
        response_assistant = get_chat_completions(conversation)
        moderation = moderation_check(response_assistant)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
        print('Please wait while we understand the problem...\n')
        ## if json present in the text, extract it
        extract_json = dictionary_present(response_assistant)
        ## check if the issue is actually found yet
        confirmation = suggest_health_solution(extract_json)
        if(confirmation is None):
            ## issue not found, ask more questions to understand the intent
            print(response_assistant)
            conversation.append({"role": "assistant", "content": str(response_assistant)})
        else:
            ## print the health solutions
            print(str(confirmation) + '\n')

In [22]:
dialogue_mgmt_system()

"Hello! I'm here to help you with your health issues. Please feel free to share any symptoms or discomfort you are experiencing so I can assist you better. What seems to be the problem?\n"

Please wait while we understand the problem...

I'm sorry to hear that you're experiencing fainting episodes. Can you provide more details about these episodes? Do you feel dizzy or lightheaded before fainting? Are there any triggers that you've noticed, such as standing up quickly or feeling extremely fatigued? Your insights will help me better understand your situation and assist you appropriately.
Please wait while we understand the problem...

Based on your symptoms of fainting, it's important to take certain measures to address the issue. To treat fainting episodes, you can follow these steps:
1. **Lay the person down:** If you feel like you're about to faint, it's essential to lie down and elevate your legs to help increase blood flow to your brain.
2. **Loosen tight clothing:** Tight clothing can restrict blood flow, so it's recommended to loosen any constrictive clothing items.
3. **Ensure good airflow:** Make sure the area where you're lying down has good ventilation to help y

#### Function calling example

In [23]:
# Define function descriptions
function_descriptions = [
    {
        "name": "list_all_department",
        "description": "Get all the departments provided by hospital.",
        "parameters": {
            "type": "object",
            "properties": {
                "department": {
                    "type": "string",
                    "description": "The required department to be shown if available"
                }
            },
            "required": ["department"]
        }
    },
    {
        "name": "get_doctors_list",
        "description": "Get list of doctors if bot can't solve a problem",
        "parameters": {
            "type": "object",
            "properties": {
                "issue": {
                    "type": "string",
                    "description": "The health issue to be solved",
                }
            },
            "required": ["issue"],
        },
    }
]

In [24]:

def get_chat_completions_functions(input):
        MODEL = 'gpt-3.5-turbo'
        chat_completion = openai_client.chat.completions.create(
            model = MODEL,
            messages = input,
            functions=function_descriptions,
            function_call="auto",
            seed = 2345)
        print(chat_completion.choices)
        output = chat_completion.choices[0].message
        return output

In [25]:
# Example input to call both functions
user_input_1 = {"role": "user", "content": "Can you tell me about the cardiology department?"}
user_input_2 = {"role": "user", "content": "I have a severe headache. Which doctor should I consult?"}


In [26]:
# Call for the first function
output_1 = get_chat_completions_functions([user_input_1])
print(output_1)

[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"department":"cardiology"}', name='list_all_department'), tool_calls=None))]
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"department":"cardiology"}', name='list_all_department'), tool_calls=None)


In [27]:
# Call for the second function
output_2 = get_chat_completions_functions([user_input_2])
print(output_2)

[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"issue":"severe headache"}', name='get_doctors_list'), tool_calls=None))]
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"issue":"severe headache"}', name='get_doctors_list'), tool_calls=None)


In [29]:
# Mock implementations of the functions
def list_all_department(department):
    return f"Department {department} is available."

def get_doctors_list(issue):
    return f"List of doctors for {issue}: Dr. A, Dr. B, Dr. C."

In [30]:
def showFunctionCallingResponse(output):
    # Extract function call details from the output and call the appropriate function
    params = json.loads(output.function_call.arguments)
    chosen_function_name = output.function_call.name
    # Map the function name to the actual function
    function_map = {
        "list_all_department": list_all_department,
        "get_doctors_list": get_doctors_list
    }

    # Call the chosen function with the extracted parameters
    if chosen_function_name in function_map:
        chosen_function = function_map[chosen_function_name]
        data = chosen_function(**params)
        print(data)
    else:
        print(f"Function {chosen_function_name} not found.")

In [31]:
showFunctionCallingResponse(output_1)

Department cardiology is available.


In [32]:
showFunctionCallingResponse(output_2)

List of doctors for severe headache: Dr. A, Dr. B, Dr. C.
